# ChatQwQ

This will help you get started with QwQ [chat models](../../concepts/chat_models.mdx). For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/).

## Overview
### Integration details


| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwQ](https://pypi.org/project/langchain-qwq/) | [langchain-qwq](https://pypi.org/project/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### Model features
| [Tool calling](../../how_to/tool_calling.ipynb) | [Structured output](../../how_to/structured_output.ipynb) | JSON mode | [Image input](../../how_to/multimodal_inputs.ipynb) | Audio input | Video input | [Token-level streaming](../../how_to/chat_streaming.ipynb) | Native async | [Token usage](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |✅  | ❌ | ✅ | ✅ | ✅ | ✅ | ❌ | 

## Setup

To access QwQ models you'll need to create an Alibaba Cloud account, get an API key, and install the `langchain-qwq` integration package.

### Credentials

Head to [Alibaba's API Key page](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en#/api-key) to sign up to Alibaba Cloud and generate an API key. Once you've done this set the `DASHSCOPE_API_KEY` environment variable:

In [1]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### Installation

The LangChain QwQ integration lives in the `langchain-qwq` package:

In [ ]:
%pip install -qU langchain-qwq

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [2]:
from langchain_qwq import ChatQwQ

llm = ChatQwQ(
    model="qwq-plus",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # other params...
)

## Invocation

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into French. Let me start by recalling the basic translation. The verb "love" in French is "aimer", and "programming" is "la programmation". So the literal translation would be "J\'aime la programmation." But wait, I should check if there\'s any context or nuances I need to consider. The user mentioned they\'re a helpful assistant, so maybe they want a more natural or commonly used phrase. Sometimes in French, people might use "adorer" instead of "aimer" for stronger emphasis, but "aimer" is more standard here. Also, the structure "J\'aime" is correct for "I love". No need for any articles if it\'s a general statement, but "la programmation" is a feminine noun, so the article is necessary. Let me confirm the gender of "programmation"—yes, it\'s feminine. So "la" is correct. I think that\'s it. The translation should be "J\'aime la programmati

## Chaining

We can [chain](../../how_to/sequence.ipynb) our model with a prompt template like so:

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', additional_kwargs={'reasoning_content': 'Okay, the user wants to translate "I love programming." into German. Let\'s start by breaking down the sentence. The subject is "I," which translates to "Ich" in German. The verb "love" is "liebe" in present tense for the first person singular. Then "programming" is a noun. Now, in German, the word for programming, especially in the context of computer programming, is "Programmierung." However, sometimes people might use "Programmieren" as well. Wait, but "Programmierung" is the noun form, so "die Programmierung." The structure in German would be "Ich liebe die Programmierung." Alternatively, could it be "Programmieren" as the verb in a nominalized form? Let me think. If you say "Ich liebe das Programmieren," that\'s also correct because "das Programmieren" is the gerundive form, which is commonly used for activities. So both are possible. Which one is more natural? Hmm. "Das Programmieren" might

## Tool Calling
ChatQwQ supports tool calling API that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### Use with `bind_tools`

In [5]:
from langchain_core.tools import tool

from langchain_qwq import ChatQwQ


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwQ()

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s 5 times forty two". Let me break this down. They want the product of 5 and 42. The function provided is called multiply, which takes two integers. First, I need to parse the numbers from the question. The first integer is 5, straightforward. The second is forty two, which is 42 in numeric form. So I should call the multiply function with first_int=5 and second_int=42. Let me double-check the parameters: both are required and of type integer. Yep, that\'s correct. No examples given, but the function should handle these numbers. Alright, time to format the tool call.'} response_metadata={'model_name': 'qwq-plus'} id='run--3c5ff46c-3fc8-4caf-a665-2405aeef2948-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_33fb94c6662d44928e56ec', 'type': 'tool_call'}] usage_metadata={'input_tokens': 176, 'output_tokens': 173, 'total_tokens': 349, 'input_token_details': {}, 'ou

### vision Support

#### Image

In [7]:
from langchain_core.messages import HumanMessage

model = ChatQwQ(model="qvq-max-latest")

messages = [
    HumanMessage(
        content=[
            {
                "type": "image_url",
                "image_url": {"url": "https://example.com/image/image.png"},
            },
            {"type": "text", "text": "What do you see in this image?"},
        ]
    )
]

response = model.invoke(messages)
print(response.content)

The image depicts a charming Christmas-themed arrangement set against a rustic wooden backdrop, creating a warm and festive atmosphere. Here's a detailed breakdown:

### **Background & Setting**
- **Wooden Wall**: A horizontally paneled wooden wall forms the backdrop, giving a cozy, cabin-like feel.
- **Foreground Surface**: The decorations rest on a smooth wooden surface (likely a table or desk), enhancing the natural, earthy tone of the scene.

### **Key Elements**
1. **Snow-Covered Trees**:
   - Two miniature evergreen trees dusted with artificial snow flank the sides of the arrangement, evoking a wintry landscape.

2. **String Lights**:
   - A strand of white bulb lights stretches across the back, weaving through the decor and adding a soft, glowing ambiance.

3. **Ornamental Sphere**:
   - A reflective gold sphere with striped patterns sits near the center-left, catching and dispersing light.

4. **"Merry Christmas" Sign**:
   - A wooden cutout spelling "MERRY CHRISTMAS" in capita

#### Video

In [10]:
from langchain_core.messages import HumanMessage

model = ChatQwQ(model="qvq-max-latest")

messages = [
    HumanMessage(
        content=[
            {
                "type": "video_url",
                "video_url": {"url": "https://example.com/video/1.mp4"},
            },
            {"type": "text", "text": "Can you tell me about this video?"},
        ]
    )
]

response = model.invoke(messages)
print(response.content)

The image provided is a still frame from a video featuring a young woman with short brown hair and bangs, smiling brightly at the camera. Here's a detailed breakdown:

### **Description of the Image:**
- **Subject:** A youthful female with a cheerful expression, showcasing a wide smile with visible teeth.
- **Appearance:** 
  - Short, neatly styled brown hair with blunt bangs.
  - Natural makeup emphasizing clear skin and subtle eye makeup.
  - Wearing a white round-neck shirt layered under a light pink knitted cardigan.
  - Accessories include a delicate necklace with a small pendant and small earrings.
- **Background:** An outdoor setting with blurred architectural elements (e.g., buildings with columns), suggesting a campus, park, or residential area.
- **Lighting:** Soft, natural daylight, enhancing the warm and inviting atmosphere.

### **Key Details About the Video:**
1. **AI-Generated Content:** The watermark ("通义·AI合成" / "Tongyi AI Synthesis") indicates this image was created u

## API reference

For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/)